# Exporting LLM Runs and Feedback
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langsmith-cookbook/blob/main/exploratory-data-analysis/exporting-llm-runs-and-feedback/llm_run_etl.ipynb)

Understanding how your LLM app interacts with users is crucial. LangSmith offers a number of useful ways to interact with and annotate trace data directly in the app. You can also easily query that trace data so you can process it with your tool of choice.

This tutorial guides you through exporting LLM traces and associated feedback from LangSmith for further analysis. By the end, you'll be able to export a flat table of LLM run information that you can analyze, enrich, or use for model training.

Before we start, ensure you have a LangChain project with some logged traces. You can generate some using almost any of the other recipes in this cookbook. The overall steps will be:

1. Query runs, filtering by time, tags, or other attributes.
2. Add in associated feedback metrics (if captured)
3. Export to analysis tool.

To make things easy, we will be loading the data into a pandas dataframe. We will be doing the ETL on LLM runs logged from LangChain, but you can modify the code below to handle whatever schema is used by your deployed model. Now let's set up our environment!

#### Setup

First, install langsmith and pandas and set your langsmith API key to connect to your project.
We will also install LangChain to use one of its formatting utilities.

In [1]:
# %pip install --upgrade --force-reinstall langchain langsmith pandas seaborn --quiet
import os  # Add this line at the beginning of your code

In [2]:
%env LANGCHAIN_API_KEY="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"
# %pip install --upgrade pandas

env: LANGCHAIN_API_KEY="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"


In [3]:
from langsmith import Client
import pandas as pd

client = Client()

## 1. Query Runs

Now that the environment is ready, we will load the run data from LangSmith. Let's try loading all our LLM runs from the past week. To do so, we will filter for runs with the "llm" `run_type` from the past week.

Please reference the [docs](https://docs.smith.langchain.com/tracing/faq/querying_traces) for guidance on more complex filters (using metadata, tags, and other attributes).


In [4]:
from datetime import datetime, timedelta
from datetime import timezone

UTC = timezone.utc
# or


start_time = datetime.now(UTC) - timedelta(days=2)
# model_converter, data_processor, sketch_generator
task_type = "model_converter"
runs = list(
    client.list_runs(
        project_name="default",
        start_time=start_time,
        end_time=datetime.now(UTC),
        run_type="chain",
        # filter=f"and(has(tags, 'gpt-4o-mini'),has(tags, {task_type}))",
        filter=f"and(has(tags, 'gpt-4o'),has(tags, {task_type}),has(tags, 'benchmark'),has(tags, '7248'))",
    )
)

In [5]:
df = pd.DataFrame(
    [
        {
            "name": run.name if run.name == task_type + "_run" else None,
            # "id": str(run.id),  # Convert UUID to string
            "latency": (
                (run.end_time - run.start_time).total_seconds()
                if run.end_time
                else None
            ),
            "time": run.start_time,
            # "tags": ', '.join(run.tags) if run.tags else None,  # Join tags into a string
            # "run_type": run.run_type,
            "num_run": run.extra["metadata"].get(
                "num_run", None
            ),  # Use .get() with a default value
            "total_tokens": run.total_tokens,
            "input_tokens": run.prompt_tokens,
            "output_tokens": run.completion_tokens,
            "total_cost": float(run.total_cost) if run.total_cost is not None else None,
            "input_cost": (
                float(run.prompt_cost) if run.prompt_cost is not None else None
            ),
            "output_cost": (
                float(run.completion_cost) if run.completion_cost is not None else None
            ),
            "pass": (
                "no"
                if "An error could not be resolved after 5 retries:" in str(run.error)
                else "yes"
            ),
            "task": task_type,
        }
        for run in runs
    ]
)

# display(df)
# print(df)

In [6]:
# display(df)
print(df.count())

name              0
latency          30
time             30
num_run          30
total_tokens     30
input_tokens     30
output_tokens    30
total_cost       30
input_cost       30
output_cost      30
pass             30
task             30
dtype: int64


In [8]:
print(os.getcwd())

/home/han/Projects/tinyml-autopilot/dev/test_in_batch/25_oct


In [9]:
import csv


def save_to_csv(df, filename=None):
    if filename is None:
        # Get the name of the current folder
        folder_name = os.getcwd().split("/")[-1]

    filename = f"{task_type}_{folder_name}.csv"
    try:
        # Convert DataFrame to a list of dictionaries manually
        data_to_save = []
        for _, row in df.iterrows():
            data_to_save.append(row.to_dict())

        with open(filename, "w", newline="") as csvfile:
            fieldnames = df.columns
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for row in data_to_save:
                writer.writerow(row)

        print(f"Data successfully saved to '{filename}'")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")


# Save the DataFrame
save_to_csv(df)

Data successfully saved to 'model_convert_25_oct.csv'


## 3. Analyze Data

Once you have the data in flat form, you can export to many compatible tools, from tabular storage like Airtable and Excel, to annotation tools like [LabelBox](https://docs.labelbox.com/reference/text-file) to other text analysis tools like [Lilac](../lilac/lilac.ipynb) or [Nomic](https://atlas.nomic.ai/).

For the purpose of this tutorial, we will wrap things up with a simple token plot. Check out our other recipes for more involved analysis.

In [10]:
date_suffix = os.getcwd().split("/")[-1]
print(date_suffix)

NameError: name '__file__' is not defined